In [1]:
from NBABase import AllStats, TodaysGames, VegasLines, TodaysPlayers

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta, date

In [3]:
columnNames = ['PlayerName', 'Position', 'Date', 'Team', 'HomeAway', 'Opp', 'Margin', 'MP', 'FG', 'FGA', 'FGper', 'ThreeP',
                    'ThreePA', 'ThreePper', 'FT', 'FTA', 'FTper', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 
                    'GmSc', 'TSPer', 'eFGPer', 'ORBPer', 'DRBPer', 'TRBPer', 'ASTPer', 'STLPer', 'BLKPer', 'TOVPer', 'USGPer', 
                    'ORTGPer', 'DRTGPer']                    
AllStats.columns = columnNames   
AllStats = AllStats.sort_values(by=['Date'])


In [10]:
data = {"Team":[],"Position":[],"Date":[],"FG":[],"FGA":[],"ThreeP":[],"ThreePA":[],"FT":[],"FTA":[],"ORB":[],"DRB":[],"TRB":[],"AST":[],"STL":[],"BLK":[],"TOV":[],"PF":[], "PTS":[],"TSPer":[],"eFGPer":[],"ORBPer":[],"DRBPer":[],"TRBPer":[],"ASTPer":[],"STLPer":[],"BLKPer":[],"TOVPer":[],"USGPer":[],"ORTGPer":[],"DRTGPer":[],"FGAOpp":[], "ThreePAOpp":[], "FTAOpp":[], "TRBOpp":[],"ASTOpp":[],"STLOpp":[],"BLKOpp":[],"TOVOpp":[]}

DefensiveStats = pd.DataFrame(data)

In [11]:
PositionsAll = ["C", "PG", "SG", "SF", "PF"]
AllTeams = list(set(AllStats.loc[:,"Team"]))
defStats_columnNames = ['Team', 'Position', 'Date', 'FG', 'FGA', 'ThreeP', 'ThreePA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TSPer', 'eFGPer', 'ORBPer', 'DRBPer', 'TRBPer', 'ASTPer', 'STLPer', 'BLKPer', 'TOVPer', 'USGPer', 'ORTGPer', 'DRTGPer', 'FGAOpp', 'ThreePAOpp', 'FTAOpp', 'TRBOpp', 'ASTOpp', 'STLOpp', 'BLKOpp', 'TOVOpp']

for team in AllTeams:    
    # date = DefensiveStats[DefensiveStats["Team"] == team][["Date"]].max()["Date"].strftime("%Y-%m-%d")
    # dateList = sorted(set(AllStats[ (AllStats["Team"] == team) & (AllStats["Date"] > date) ]["Date"]))
    dateList = sorted(set(AllStats[(AllStats["Team"] == team)]["Date"]))
    
    print(team)
    for date in dateList:        
        for pos in PositionsAll:            
            temp = pd.DataFrame(index=range(0,1), columns = defStats_columnNames)
            dataSubset = AllStats[(AllStats["Team"] == team) &
                                  (AllStats["Position"] == pos) & 
                                  (AllStats["Date"] < date) & 
                                  (AllStats["Date"] > (date + pd.Timedelta(days=-300)) )]
            dataSubset = dataSubset.sort_values(by = "Date", ascending = True)
            currentGame = AllStats[(AllStats["Team"] == team) & (AllStats["Date"] == date)]

            if (len(currentGame["Date"]) == 0) or (len(dataSubset["Date"]) == 0):
                break
            
            if (len(dataSubset) > 50):
                dataSubset = dataSubset[0:50]
            
            temp["Date"] = date
            temp["Team"] = team
            temp["Position"] = pos
            #### How does team perform in this position historically
            for i in defStats_columnNames[3:30]:
                temp[i] = np.mean(AllStats[i])
            
            ### Get Opposition Players in the game
            OppPlayers = list(AllStats[(AllStats["Team"].isin(list(currentGame["Opp"][0:1])) == True) &
                                  (AllStats["Position"] == pos) & 
                                  (AllStats["Date"] == date)]["PlayerName"])

            ### Get Opposition Players and their dates to find all games these players particiated in      
            OppDates = sorted(list(AllStats[AllStats["PlayerName"].isin(OppPlayers) == True]["Date"]), reverse = True)
            OppDates = OppDates[0:5]
          
            ## Make sure do not include this date but eveytyhing before
            subsetOppData =  AllStats[(AllStats["Team"].isin(list(currentGame["Opp"][0:1])) == True) &
                                  (AllStats["PlayerName"].isin(OppPlayers) == True) &
                                  (AllStats["Date"].isin(OppDates) == True)]

            subsetOppData = subsetOppData.sort_values(by = "Date", ascending = True)
            if (len(subsetOppData) > 50):
                subsetOppData = subsetOppData[0:50]
            
            #### How does team perform in this position historically
            for y in defStats_columnNames[30:38]:         
                temp[y] = np.mean(subsetOppData[y.replace("Opp", "")])

            DefensiveStats = DefensiveStats.append(temp, ignore_index = True)

NOP
DEN
SAC
TOR
ATL
LAC
CLE
CHO
LAL
CHI
DET
POR
SAS
BOS
UTA
ORL
PHO
MIN
BRK
HOU
DAL
MIL
WAS
IND
MIA
NYK
GSW
MEM
OKC
PHI


In [13]:
DefensiveStats.to_csv("DefensiveStats.csv" , index=False)


function CalcuateDefensiveStats(defStats):
    for team in AllTeams:    
        date = DefensiveStats[DefensiveStats["Team"] == team][["Date"]].max()["Date"].strftime("%Y-%m-%d")
        dateList = sorted(set(AllStats[(AllStats["Team"] == team)]["Date"]))
        dateList = sorted(set(AllStats[ (AllStats["Team"] == team) & (AllStats["Date"] > date) ]["Date"]))
        
    
        print(team)
        for date in dateList:        
            for pos in PositionsAll:            
                temp = pd.DataFrame(index=range(0,1), columns = defStats_columnNames)
                dataSubset = AllStats[(AllStats["Team"] == team) &
                                    (AllStats["Position"] == pos) & 
                                    (AllStats["Date"] < date) & 
                                    (AllStats["Date"] > (date + pd.Timedelta(days=-300)) )]
                dataSubset = dataSubset.sort_values(by = "Date", ascending = True)
                currentGame = AllStats[(AllStats["Team"] == team) & (AllStats["Date"] == date)]

                if (len(currentGame["Date"]) == 0) or (len(dataSubset["Date"]) == 0):
                    break
                
                if (len(dataSubset) > 50):
                    dataSubset = dataSubset[0:50]
                
                temp["Date"] = date
                temp["Team"] = team
                temp["Position"] = pos
                #### How does team perform in this position historically
                for i in defStats_columnNames[3:30]:
                    temp[i] = np.mean(AllStats[i])
                
                ### Get Opposition Players in the game
                OppPlayers = list(AllStats[(AllStats["Team"].isin(list(currentGame["Opp"][0:1])) == True) &
                                    (AllStats["Position"] == pos) & 
                                    (AllStats["Date"] == date)]["PlayerName"])

                ### Get Opposition Players and their dates to find all games these players particiated in      
                OppDates = sorted(list(AllStats[AllStats["PlayerName"].isin(OppPlayers) == True]["Date"]), reverse = True)
                OppDates = OppDates[0:5]
            
                ## Make sure do not include this date but eveytyhing before
                subsetOppData =  AllStats[(AllStats["Team"].isin(list(currentGame["Opp"][0:1])) == True) &
                                    (AllStats["PlayerName"].isin(OppPlayers) == True) &
                                    (AllStats["Date"].isin(OppDates) == True)]

                subsetOppData = subsetOppData.sort_values(by = "Date", ascending = True)
                if (len(subsetOppData) > 50):
                    subsetOppData = subsetOppData[0:50]
                
                #### How does team perform in this position historically
                for y in defStats_columnNames[30:38]:         
                    temp[y] = np.mean(subsetOppData[y.replace("Opp", "")])

                defStats = defStats.append(temp, ignore_index = True)      

In [23]:
team

date = DefensiveStats[DefensiveStats["Team"] == team][["Date"]].max()["Date"].strftime("%Y-%m-%d")
date
dateList = sorted(set(AllStats[(AllStats["Team"] == team)]["Date"]))
dateList = sorted(set(AllStats[ (AllStats["Team"] == team) & (AllStats["Date"] > date) ]["Date"]))
